In [1]:
# Import Packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [4]:
#Load the data set
#top 30 cyptocurrencies on yahoo api. columns represent stock prices on ADJ close
stock_prices = pd.read_csv('./Data/Yahoo_Stock_Prices.csv')

In [5]:
stock_prices

,Date,AAVE-USD,ADA-USD,ALGO-USD,AMP1-USD,BCH-USD,BNB-USD,BTC-USD,CRO-USD,DOGE-USD,...,SOL1-USD,THETA-USD,TRX-USD,UNI3-USD,USDC-USD,USDT-USD,VET-USD,XLM-USD,XMR-USD,XRP-USD
0,2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN,314.248993,NaN,0.000183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005493,0.465760,0.024390
1,2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,315.032013,NaN,0.000186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005481,0.459137,0.024318
2,2015-01-03,NaN,NaN,NaN,NaN,NaN,NaN,281.082001,NaN,0.000167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004672,0.461172,0.022106
3,2015-01-04,NaN,NaN,NaN,NaN,NaN,NaN,264.195007,NaN,0.000158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004416,0.417234,0.018928
4,2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,274.473999,NaN,0.000161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004947,0.421412,0.020316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2338,2021-05-28,320.381165,1.511987,0.898697,0.036657,689.947876,329.879913,35697.605469,0.113855,0.311205,...,29.039276,6.564076,0.072087,25.924946,1.000196,1.000481,0.108300,0.398792,262.820526,0.900148
2339,2021-05-29,297.899628,1.406232,0.829413,0.033361,651.579956,305.052155,34616.066406,0.111285,0.303110,...,27.376421,6.313582,0.068783,23.179464,0.999901,1.000392,0.098990,0.367157,242.684479,0.831788
2340,2021-05-30,325.148407,1.577279,0.850180,0.037683,663.512024,325.784027,35678.128906,0.112185,0.302224,...,28.602659,6.545633,0.072036,25.742519,0.999498,1.000063,0.107572,0.375688,262.333740,0.906609
2341,2021-05-31,383.083405,1.742778,0.937608,0.037412,703.882568,354.329346,37332.855469,0.125617,0.325810,...,32.816246,7.127231,0.076920,28.369711,1.000264,1.000533,0.126158,0.403407,272.919495,1.046584
